# ChatOCIModelDeployment

这将帮助您开始使用 OCIModelDeployment [聊天模型](/docs/concepts/chat_models)。有关 ChatOCIModelDeployment 功能和配置的详细文档，请参阅 [API 参考](https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.oci_data_science.ChatOCIModelDeployment.html)。

[OCI Data Science](https://docs.oracle.com/en-us/iaas/data-science/using/home.htm) 是一个完全托管的无服务器平台，供数据科学团队在 Oracle Cloud Infrastructure 中构建、训练和管理机器学习模型。您可以使用 [AI Quick Actions](https://blogs.oracle.com/ai-and-datascience/post/ai-quick-actions-in-oci-data-science) 在 [OCI Data Science 模型部署服务](https://docs.oracle.com/en-us/iaas/data-science/using/model-dep-about.htm)上轻松部署 LLM。您可以选择使用流行的推理框架（如 vLLM 或 TGI）来部署模型。默认情况下，模型部署端点模仿 OpenAI API 协议。

> 有关最新更新、示例和实验性功能，请参阅 [ADS LangChain 集成](https://accelerated-data-science.readthedocs.io/en/latest/user_guide/large_language_model/langchain_models.html)。

## 概述
### 集成详情

| 类 | 包 | 本地 | 可序列化 | JS 支持 | 包下载 | 包最新 |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [ChatOCIModelDeployment](https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.oci_data_science.ChatOCIModelDeployment.html) | [langchain-community](https://python.langchain.com/api_reference/community/index.html) | ❌ | beta | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-community?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-community?style=flat-square&label=%20) |

### 模型功能

| [工具调用](/docs/how_to/tool_calling) | [结构化输出](/docs/how_to/structured_output/) | JSON 模式 | [图像输入](/docs/how_to/multimodal_inputs/) | 音频输入 | 视频输入 | [令牌级流式传输](/docs/how_to/chat_streaming/) | 原生异步 | [令牌使用情况](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| 依赖 | 依赖 | 依赖 | 依赖 | 依赖 | 依赖 | ✅ | ✅ | ✅ | ✅ |

某些模型功能，包括工具调用、结构化输出、JSON 模式和多模态输入，取决于已部署的模型。

## 设置

要使用 ChatOCIModelDeployment，您需要部署一个具有聊天完成端点的聊天模型，并安装 `langchain-community`、`langchain-openai` 和 `oracle-ads` 集成包。

您可以使用 OCI Data Science 模型部署上的 [AI Quick Actions](https://github.com/oracle-samples/oci-data-science-ai-samples/blob/main/ai-quick-actions/model-deployment-tips.md) 轻松部署基础模型。有关其他部署示例，请访问 [Oracle GitHub 示例存储库](https://github.com/oracle-samples/oci-data-science-ai-samples/tree/main/ai-quick-actions)。

### 策略
请确保您拥有访问 OCI Data Science 模型部署端点所需的[策略](https://docs.oracle.com/en-us/iaas/data-science/using/model-dep-policies-auth.htm#model_dep_policies_auth__predict-endpoint)。

### 凭证

您可以通过 Oracle ADS 设置身份验证。当您在 OCI Data Science Notebook Session 中工作时，可以利用资源主体来访问其他 OCI 资源。

In [ ]:
import ads

# Set authentication through ads
# Use resource principal are operating within a
# OCI service that has resource principal based
# authentication configured
ads.set_auth("resource_principal")

或者，您可以使用以下环境变量配置凭据。例如，要使用具有特定配置文件的 API 密钥：

In [ ]:
import os

# Set authentication through environment variables
# Use API Key setup when you are working from a local
# workstation or on platform which does not support
# resource principals.
os.environ["OCI_IAM_TYPE"] = "api_key"
os.environ["OCI_CONFIG_PROFILE"] = "default"
os.environ["OCI_CONFIG_LOCATION"] = "~/.oci"

请查看 [Oracle ADS 文档](https://accelerated-data-science.readthedocs.io/en/latest/user_guide/cli/authentication.html) 以了解更多选项。

### 安装

LangChain OCIModelDeployment 集成位于 `langchain-community` 包中。以下命令将安装 `langchain-community` 和所需的依赖项。

In [ ]:
%pip install -qU langchain-community langchain-openai oracle-ads

## 实例化

您可以使用通用的 `ChatOCIModelDeployment` 或特定框架的类，如 `ChatOCIModelDeploymentVLLM` 来实例化模型。

* 当您需要部署模型的通用入口点时，请使用 `ChatOCIModelDeployment`。您可以在实例化此类时通过 `model_kwargs` 传递模型参数。这提供了灵活性和便捷的配置，而无需依赖特定框架的细节。

In [ ]:
from langchain_community.chat_models import ChatOCIModelDeployment

# Create an instance of OCI Model Deployment Endpoint
# Replace the endpoint uri with your own
# Using generic class as entry point, you will be able
# to pass model parameters through model_kwargs during
# instantiation.
chat = ChatOCIModelDeployment(
    endpoint="https://modeldeployment.<region>.oci.customer-oci.com/<ocid>/predict",
    streaming=True,
    max_retries=1,
    model_kwargs={
        "temperature": 0.2,
        "max_tokens": 512,
    },  # other model params...
    default_headers={
        "route": "/v1/chat/completions",
        # other request headers ...
    },
)

* 使用特定框架的类，如 `ChatOCIModelDeploymentVLLM`：当您使用特定框架（例如 `vLLM`）并需要直接通过构造函数传递模型参数时，这非常适用，可以简化设置过程。

In [ ]:
from langchain_community.chat_models import ChatOCIModelDeploymentVLLM

# Create an instance of OCI Model Deployment Endpoint
# Replace the endpoint uri with your own
# Using framework specific class as entry point, you will
# be able to pass model parameters in constructor.
chat = ChatOCIModelDeploymentVLLM(
    endpoint="https://modeldeployment.<region>.oci.customer-oci.com/<md_ocid>/predict",
)

## 调用

In [2]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

ai_msg = chat.invoke(messages)
ai_msg

AIMessage(content="J'adore programmer.", response_metadata={'token_usage': {'prompt_tokens': 44, 'total_tokens': 52, 'completion_tokens': 8}, 'model_name': 'odsc-llm', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='run-ca145168-efa9-414c-9dd1-21d10766fdd3-0')

In [3]:
print(ai_msg.content)


J'adore programmer.


## Chaining

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe Programmierung.', response_metadata={'token_usage': {'prompt_tokens': 38, 'total_tokens': 48, 'completion_tokens': 10}, 'model_name': 'odsc-llm', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='run-5dd936b0-b97e-490e-9869-2ad3dd524234-0')

## 异步调用

In [6]:
from langchain_community.chat_models import ChatOCIModelDeployment

system = "You are a helpful translator that translates {input_language} to {output_language}."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chat = ChatOCIModelDeployment(
    endpoint="https://modeldeployment.us-ashburn-1.oci.customer-oci.com/<ocid>/predict"
)
chain = prompt | chat

await chain.ainvoke(
    {
        "input_language": "English",
        "output_language": "Chinese",
        "text": "I love programming",
    }
)

AIMessage(content='我喜欢编程', response_metadata={'token_usage': {'prompt_tokens': 37, 'total_tokens': 50, 'completion_tokens': 13}, 'model_name': 'odsc-llm', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='run-a2dc9393-f269-41a4-b908-b1d8a92cf827-0')

## 流式通话

In [7]:
import os
import sys

from langchain_community.chat_models import ChatOCIModelDeployment
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("human", "List out the 5 states in the United State.")]
)

chat = ChatOCIModelDeployment(
    endpoint="https://modeldeployment.us-ashburn-1.oci.customer-oci.com/<ocid>/predict"
)

chain = prompt | chat

for chunk in chain.stream({}):
    sys.stdout.write(chunk.content)
    sys.stdout.flush()



1. California
2. Texas
3. Florida
4. New York
5. Illinois

## 结构化输出

In [8]:
from langchain_community.chat_models import ChatOCIModelDeployment
from pydantic import BaseModel


class Joke(BaseModel):
    """A setup to a joke and the punchline."""

    setup: str
    punchline: str


chat = ChatOCIModelDeployment(
    endpoint="https://modeldeployment.us-ashburn-1.oci.customer-oci.com/<ocid>/predict",
)
structured_llm = chat.with_structured_output(Joke, method="json_mode")
output = structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)

output.dict()

{'setup': 'Why did the cat get stuck in the tree?',
 'punchline': 'Because it was chasing its tail!'}

## API 参考

有关所有功能和配置的详细信息，请参阅每个类的 API 参考文档：

* [ChatOCIModelDeployment](https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.oci_data_science.ChatOCIModelDeployment.html)
* [ChatOCIModelDeploymentVLLM](https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.oci_data_science.ChatOCIModelDeploymentVLLM.html)
* [ChatOCIModelDeploymentTGI](https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.oci_data_science.ChatOCIModelDeploymentTGI.html)